# EDA - EEG data

Checl the class distributions

In [1]:
import numpy as np
import pandas as pd
import os
from readers_preprocess import cut_sequences
import matplotlib
import matplotlib.pyplot as plt
%matplotlib notebook


In [2]:

all_paths = ['data_bci\\row_data\\subject1\\', 'data_bci\\row_data\\subject2\\','data_bci\\row_data\\subject3\\']
keep_columns = ['F4', 'F3', 'AF4', 'AF3', 'class']
columns_to_read =  ['Fp1', 'AF3' ,'F7', 'F3', 'FC1', 'FC5', 'T7', 'C3', 'CP1', 'CP5',
                   'P7', 'P3', 'Pz', 'PO3', 'O1', 'Oz', 'O2', 'PO4', 'P4', 'P8', 'CP6',
                   'CP2', 'C4', 'T8', 'FC6', 'FC2', 'F4', 'F8', 'AF4', 'Fp2', 'Fz', 'Cz','class']


In [3]:
def read_data(all_paths, columns_to_read, cutoff_beggining, seq_len, cut_step):
    map_dict = {2:0, 3:1, 7:2}

    columns = ['Fp1', 'AF3' ,'F7', 'F3', 'FC1', 'FC5', 'T7', 'C3', 'CP1', 'CP5',
               'P7', 'P3', 'Pz', 'PO3', 'O1', 'Oz', 'O2', 'PO4', 'P4', 'P8', 'CP6',
               'CP2', 'C4', 'T8', 'FC6', 'FC2', 'F4', 'F8', 'AF4', 'Fp2', 'Fz', 'Cz','class']
    
    session_1_mark = '01'
    session_2_mark = '02'
    session_3_mark = '03'
    

    data = []
    for path in all_paths:
        files = os.listdir(path)
        
        for file in files:
            if 'train' not in file:
                continue
            ascii_grid = np.loadtxt(path+"\\"+file)
            experiment_data = pd.DataFrame(ascii_grid, columns=columns)
            experiment_data = experiment_data[columns_to_read]
            diffs =(experiment_data['class'] != experiment_data['class'].shift(1))
            experiment_data['instance_id'] = diffs.cumsum()
            experiment_data['class'] = experiment_data['class'].apply(lambda x : map_dict[x])
            experiment_data_seqs, annotations = cut_sequences(experiment_data, columns_to_read[:-1], cutoff_beggining, seq_len, cut_step)
            if session_1_mark in file:
                session_1_data = experiment_data_seqs
                session_1_anots = annotations
            elif session_2_mark in file:
                session_2_data = experiment_data_seqs
                session_2_anots = annotations
            elif session_3_mark in file:

                session_3_data = experiment_data_seqs
                session_3_anots = annotations
        data.append(((session_1_data, session_2_data, session_3_data),(session_1_anots, session_2_anots, session_3_anots)))
    return data

In [ ]:
all_paths = ['data_bci\\row_data\\subject1\\', 'data_bci\\row_data\\subject2\\','data_bci\\row_data\\subject3\\']
keep_columns = ['F4', 'F3', 'AF4', 'AF3', 'class']
columns_to_read =  ['Fp1', 'AF3' ,'F7', 'F3', 'FC1', 'FC5', 'T7', 'C3', 'CP1', 'CP5',
                   'P7', 'P3', 'Pz', 'PO3', 'O1', 'Oz', 'O2', 'PO4', 'P4', 'P8', 'CP6',
                   'CP2', 'C4', 'T8', 'FC6', 'FC2', 'F4', 'F8', 'AF4', 'Fp2', 'Fz', 'Cz','class']




freq = 512

cutoff_beggining = 0
seq_len = 0
cut_step = 0
data = read_data(all_paths, columns_to_read, cutoff_beggining, seq_len, cut_step)
seq_len = freq
cut_step = freq
short_data = read_data(all_paths, columns_to_read, cutoff_beggining, seq_len, cut_step)
   

In [ ]:
def eval_sess(X, y, freq=512):
    
    time_len_dict = {}
    num_instances_dict = {}
    class_map = {0:'left',1:'right',2:'word'}
    for class_num, signal in zip(y, X):
        
        class_anot = class_map[class_num]
        if class_anot not in time_len_dict:
            time_len_dict[class_anot] = signal.shape[0]/freq
            num_instances_dict[class_anot] = 1
        else:
            time_len_dict[class_anot] += signal.shape[0]/freq
            num_instances_dict[class_anot] += 1

    return time_len_dict, num_instances_dict
 

def eval_data(data):
    
    perssons_time_data = []
    perssons_num_data = []
    for person_num in range(len(data)):
        person_time_dict = {}
        person_num_inst_dict = {}
        person_data = data[person_num]
        for sess_X, sess_y in zip(person_data[0], person_data[1]):
            time_len_dict, num_inst_dict = eval_sess(sess_X, sess_y)
            num_word = num_inst_dict['word']
            num_left = num_inst_dict['left']
            num_right = num_inst_dict['right']
            len_word = time_len_dict['word']
            len_left = time_len_dict['left']
            len_right  = time_len_dict['right']
            
            if 'word' not in person_time_dict:
                person_num_inst_dict['word'] = [num_word]
                person_num_inst_dict['left'] = [num_left]
                person_num_inst_dict['right'] = [num_right]
                
                person_time_dict['word'] = [len_word]
                person_time_dict['left'] = [len_left]
                person_time_dict['right'] = [len_right]
            else:
                person_num_inst_dict['word'].append(num_word)
                person_num_inst_dict['left'].append(num_left)
                person_num_inst_dict['right'].append(num_right)
                
                person_time_dict['word'].append(len_word)
                person_time_dict['left'].append(len_left)
                person_time_dict['right'].append(len_right)

        perssons_num_data.append(person_num_inst_dict)
        perssons_time_data.append(person_time_dict)
    return perssons_num_data,perssons_time_data
perssons_num_data, perssons_time_data = eval_data(data)

In [ ]:
def plot_report(reports, y_labels, tittles):
    
    def autolabel(rects, xpos='center'):
        """
        Attach a text label above each bar in *rects*, displaying its height.

        *xpos* indicates which side to place the text w.r.t. the center of
        the bar. It can be one of the following {'center', 'right', 'left'}.
        """

        ha = {'center': 'center', 'right': 'left', 'left': 'right'}
        offset = {'center': 0, 'right': 1, 'left': -1}

        for rect in rects:
            height = rect.get_height()
            ax.annotate('{}'.format(height),
                        xy=(rect.get_x() +rect.get_width() / 3, height),
                        xytext=(offset[xpos]*3, 3),  # use 3 points offset
                        textcoords="offset points",  # in both directions
                        ha=ha[xpos], va='bottom')
    
    fig, axes = plt.subplots(1, 2, figsize=(8, 4))
    axes = axes.flatten()
    
    for i in range(2):
        ax = axes[i]
        report = reports[i]
        y_label = y_labels[i]
        tittle = tittles[i]
    

        word = [int(el) for el in report['word']]
        left = [int(el) for el in report['left']]
        right = [int(el) for el in report['right']]

        ind = np.arange(len(word))  # the x locations for the groups
        width = 0.5  # the width of the bars

        rects1 = ax.bar(ind - width/3, word, width/3,label='word')

        rects2 = ax.bar(ind, left, width/3, label='left')

        rects3 = ax.bar(ind + width/3, right, width/3, label='right')
        # Add some text for labels, title and custom x-axis tick labels, etc.
        ax.set_ylabel(y_label)
        ax.set_title(tittle)
        ax.set_xticks(ind)
        ax.set_xticklabels(('session1', 'session2', 'session3'))
        ax.legend()

        if i==0:
            autolabel(rects1, "left")
            autolabel(rects2, "center")
            autolabel(rects3, "right")

        ax.legend(fancybox=True, framealpha=0.3)


In [ ]:
person1_tittle = 'Volunteer 1'
person2_tittle = 'Volunteer 2'
person3_tittle = 'Volunteer 3'


time_y_label = 'Tottal signal lenght [s]'
num_insts_y_label = 'Tottal task repetitions'

In [ ]:
person1_time = perssons_time_data[0]
person1_num_insts = perssons_num_data[0]
person2_time = perssons_time_data[1]
person2_num_insts = perssons_num_data[1]
person3_time = perssons_time_data[2]
person3_num_insts = perssons_num_data[2]

In [ ]:

plot_report([person1_time, person1_num_insts],[ time_y_label, num_insts_y_label], [person1_tittle, person1_tittle])

In [ ]:
plot_report([person2_time, person2_num_insts],[ time_y_label, num_insts_y_label], [person2_tittle, person2_tittle])

In [ ]:
plot_report([person3_time, person3_num_insts],[ time_y_label, num_insts_y_label], [person3_tittle, person3_tittle])